This demonstrates streaming data from a second thread and periodically updating the plot.

- source.stream(self.df) will eventually bog down, as df size grows, so limit n_show

In [30]:
import pandas as pd
import numpy as np

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.application.handlers.handler import Handler
from bokeh.application.application import Application
from bokeh.server.server import Server
from bokeh.io import output_notebook, show
from threading import Lock

class PeriodicDataframeHandler(Handler):
    def __init__(self):
        super(PeriodicDataframeHandler, self).__init__()
        self.period_ms = 500
        self.update = False
        self.df = pd.DataFrame(columns=['iter', 'cost'])
        self.reset = False
        self.lock = Lock()
        self.n_show = 1000
        
    def modify_document(self, doc):
        self.source = ColumnDataSource(self.df)
        p = figure()
        p.line('iter', 'cost', source=self.source)
        def callback():
            with self.lock:
                if self.reset:
                    self.reset = False
                    self.source.data = ColumnDataSource.from_df(self.df)
                if self.update:
                    self.source.stream(self.df, self.n_show)
                    self.update = False

        doc.add_root(p)
        doc.add_periodic_callback(callback, self.period_ms)

    def add_data(self, new_df_or_update):
        with self.lock:
            self.df.loc[len(self.df)] = new_df_or_update
            self.update = True
        
    def reset_data(self):
        with self.lock:
            self.df = pd.DataFrame(columns=['iter', 'cost'])
            self.reset = True
        
output_notebook()
handler = PeriodicDataframeHandler()
bkapp = Application(handler)
show(bkapp, notebook_url='http://localhost:8890')

Loading BokehJS ...

In [33]:
import time
from threading import Thread

stop_threads = False

def mock_train(stop):
    
    for i in range(100000):
        noise = np.random.uniform(-1,1)
        cost = 100 * (0.99**(i/10.))
        cost += np.random.uniform(-cost/10,cost/10)
        handler.add_data(dict(iter=i, cost=cost))
        time.sleep(0.01)
        if stop():
            break
        
thread = Thread(target=mock_train, args=(lambda: stop_threads, ))
thread.start()

In [32]:
stop_threads = True
thread.join()
handler.reset_data()
del thread